# cuda-convnet实现CIFAR-10的识别

## 网络结构图

| Layer名称 | 描述 |
|:----------|:------|
|conv1 | 卷积层和ReLU激活函数 |
|pool1 | 最大池化层 |
|norm1 | LRN |
|conv2 | 卷积层和ReLU激活函数 |
|norm2 | LRN |
|pool2 | 最大池化层 |
|local3 | 全连接层和ReLU激活函数 |
|local4 | 全连接层和ReLU激活函数 |
|logist | 模型Inference的输出结果 |

## 特点
- 对weights进行L2的正则化
- 对图片进行了翻转随机剪切等数据增强，制造更多样本
- 在每个卷积层-最大池化层后面使用LRN层，增强模型泛化能力

In [1]:
# 载入库文件
import cifar10, cifar10_input
import tensorflow as tf
import numpy as np
import time
import os
import sys
import tarfile
import re
from six.moves import urllib

In [2]:
# 定义常用参数
max_steps = 30 # 最大轮数
batch_size = 30 # batch_size
data_dir = '../../datasets/cifar10/' # 下载cifar10数据的默认路径
DATA_URL = 'https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz'

In [3]:
# 初始化weight函数
# 一般说来，L1正则会制造稀疏的特征，大部分无用特征的权重会被置0，
# 而L2正则会让特征的权重不过大,使得特征的权重比较平均
# 我们使用tf.nn.l2_loss函数计算weight的L2 loss
# 再使用tf.multiply让L2 loss乘以w1，得到最后的weights loss
# 我们接着使用tf.add_to_collection把weights统一存到一个collection钟，这个collection名为'losses'
def variable_with_weight_loss(shape, stddev, w1):
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if w1 is not None:
        weigth_loss = tf.multiply(tf.nn.l2_loss(var), w1, name='weight_loss')
        tf.add_to_collection('losses', weigth_loss)
    return var

In [4]:
def maybe_download_and_extract():
  """Download and extract the tarball from Alex's website."""
  dest_directory = data_dir
  if not os.path.exists(dest_directory):
    os.makedirs(dest_directory)
  filename = DATA_URL.split('/')[-1]
  filepath = os.path.join(dest_directory, filename)
  if not os.path.exists(filepath):
    def _progress(count, block_size, total_size):
      sys.stdout.write('\r>> Downloading %s %.1f%%' % (filename,
          float(count * block_size) / float(total_size) * 100.0))
      sys.stdout.flush()
    filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath, _progress)
    print()
    statinfo = os.stat(filepath)
    print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
  extracted_dir_path = os.path.join(dest_directory, 'cifar-10-batches-bin')
  if not os.path.exists(extracted_dir_path):
    tarfile.open(filepath, 'r:gz').extractall(dest_directory)

In [5]:
# 下载cifar10类数据集，并解压，展开到其默认位置
# maybe_download_and_extract()


# 在使用cifar10_input类中的distorted_inputs函数产生训练需要的数据集，包括特征对应
# 的label，这里返回的是以及封装好的tensor,每次执行都会产生一个batch_size数量的样本。
# 需要注意的是我们对数据进行了Data Augmentation(数据增强)。具体的实现细节，读者可以
# 查看cifar10_input.distored_inputs函数，其中的数据增强操作包括随机的水平翻转
# (tf.image.random_flip_left_right)，随机剪切一块24*24大小的图片(tf.random_crop),
# 设置随机的亮度和对比度(tf.image.random_brightness, tf.image.random_contrast).
# 以及对数据进行标准化(tf.image.per_image_whitening, 对数据减去均值，除以方差，保证数据零均值，方差为1)
# 通过这些操作，我们可以获得更多的样本(带噪声的)，原来的一张图片样本可以变为多张图片
# 相当于扩大了样本容量，对提高准确率非常有帮助。
# 需要主要的是，我们队图像进行数据增强的操作时需要耗费大量CPU时间，因此distorted_inputs
# 使用16个独立线程来加速任务，函数内部会产生线程池，在需要使用时会通过TensorFlow queue进行调度
data_dir = os.path.join(data_dir, 'cifar-10-batches-bin')
images_train, labels_train = cifar10_input.distorted_inputs(data_dir=data_dir, batch_size=batch_size)

# 我们再使用cifar10_input.inputs函数生成测试数据，这里不需要进行太多处理，不需要对图片
# 进行翻转或者修改亮度，对比度，不过需要剪裁图片正中间的24*24大小的区块，进行数据标准化操作
images_test, labels_test = cifar10_input.inputs(eval_data=True,
                                              data_dir=data_dir,
                                              batch_size=batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [6]:
# 这里创建输入数据的placeholder，包括特征和label。
# 在设定placeholder尺寸时需要注意，因为batch_size在之后定义网络结构时被用到。
# 所以数据尺寸中的第一个值集样本条数需要预先设定，而不是像以前一样可以设定为None
# 而数据尺寸中的图片为24*24，即剪裁后的大小，而颜色通道则设为3,代表图片是彩色RGB三
# 三通道
image_holder = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])
label_holder = tf.placeholder(tf.int32, [batch_size])

In [7]:
# 开始创建第一个卷积层
# 第一个卷积层使用了5*5的卷积核,3个颜色通道，64个卷积核,
# 同时设置weights初始化函数的标准差为0.05;
# 我们不对第一个卷积层weight进行L2的正则，因此w1(weight loss)这一项设为0
# 使用tf.nn.conv2d函数对输入数据image_holder进行卷积操作，这里的步长stride均设为1，
# padding模式为SAME。把这一层的bias全部设为0，再将卷积的结果加上bias
# 最后使用ReLU激活函数进行非线性化。
# 在ReLU激活函数之后，我们使用一个尺寸为3*3且步长为2*2的最大池化层处理数据，注意这里
# 这里最大池化层的尺寸和步长不一致，这样可以增加数据的丰富性。
# 再之后，使用tf.nn.lrn函数，即LRN对结果进行处理。LRN最早见于Alex那篇CNN参加ImageNet
# 比赛的论文，Alex在论文中解释LRN层模仿了生物神经系统的“侧抑制”机制，对局部神经元的活动
# 创造竞争环境，使得其中响应比较大的值变得相对更大，并抑制其他反馈比较小的神经元，增强了
# 模型的泛化能力
# Alex在ImageNet数据集上的实现表明，使用LRN后CNN在top1的错误率可以降低1.4%,因此Alex在其
# 经典的AlexNet中使用了LRN层。LRN层对ReLU种没有上限边界的激活函数会比较有用，因为它会从
# 附近的多个卷积核的响应(Response)中挑选比较大的反馈，但是不适合Sigmiod这种有固定边界并且
# 能抑制过大值的激活函数。
weight1 = variable_with_weight_loss(shape=[5, 5, 3, 64], stddev=5e-2, w1=0.0)
kernel1 = tf.nn.conv2d(image_holder, weight1, [1, 1, 1, 1], padding='SAME')
bias1 = tf.Variable(tf.constant(0.0, shape=[64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernel1, bias1))
pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
norml = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)

In [8]:
# 创建第二个卷积层.和第一卷积层步骤很想
# 本层卷积层的尺寸也为64
# bias全部为0.1
# 最后调换最大池化层和LRN层的顺序，先进行LRN层处理，再进行池化层处理
weigth2 = variable_with_weight_loss(shape=[5, 5, 64, 64], stddev=5e-2, w1=0.0)
kernel2 = tf.nn.conv2d(norml, weigth2, [1, 1, 1, 1], padding='SAME')
bias2 = tf.Variable(tf.constant(0.1, shape=[64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))
norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)
pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

In [9]:
# 创建全连接层
# 隐含层个数为384
reshape = tf.reshape(pool2, [batch_size, -1])
dim = reshape.get_shape()[1].value
weight3 = variable_with_weight_loss([dim, 384], stddev=0.04, w1=0.04)
bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)

In [10]:
# 创建全连接层，隐含层个数192
weight4 = variable_with_weight_loss([384, 192], stddev=0.04, w1=0.04)
bias4 = tf.Variable(tf.constant(0.1, shape=[192]))
local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)

In [11]:
# 最后一层硬汉层
weight5 = variable_with_weight_loss([192, 10], stddev=1/192.0, w1=0.0)
bias5 = tf.Variable(tf.constant(0.1,shape=[10]))
logits = tf.add(tf.matmul(local4, weight5), bias5)

In [12]:
# 定义loss函数
def loss(logits, labels):
    # 计算结果的交叉熵
    tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=labels, name = "cross_entropy_per_example")
    # 计算结果交叉熵的均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    # 把交叉熵的均值加入'loss'集合
    tf.add_to_collection('losses', cross_entropy_mean)
    # 返回'loss'集合中的损失和
    return tf.add_n(tf.get_collection('losses'), name='total_loss')

In [13]:
loss = loss(logits=logits, labels=label_holder)

In [14]:
# 定义优化器
train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [15]:
# 输出top-k的正确率
top_k_op = tf.nn.in_top_k(logits, label_holder, 1)

In [16]:
# 创建默认的session
sess = tf.InteractiveSession()

# 初始化所有变量
tf.global_variables_initializer().run()

In [17]:
# 启动图片数据增强的线程队列
tf.train.start_queue_runners()

[<Thread(QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany, started daemon 1312)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 10232)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 6984)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 1748)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 9404)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 9732)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 2392)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started dae

In [ ]:
# 开始进行训练
for step in range(max_steps):
    print("step: %d" % step)
    start_time = time.time()
    # 获取训练数据
    image_batch, label_batch = sess.run([images_train, labels_train])
    # 进行训练
    _, loss_value = sess.run([train_op, loss], 
                             feed_dict={image_holder: image_batch, 
                                        label_holder: label_batch})
    # 计算训练一个batch耗费的时间
    duration = time.time() - start_time
    
    # 每10步打印一次数据
    if step % 10 == 0:
        examples_per_sec = batch_size / duration
        sec_per_batch = float(duration)
        
        format_str = ('step %d, loss=%.2f (%.1f examples/sec; %.3f sec/batch)')
        print(format_str % (step, loss_value, examples_per_sec, sec_per_batch))

step: 0


In [ ]:
# 评测模型在测试集上的准确率
num_examples = 10000
import math
num_iter = int(math.ceil(num_examples / batch_size))
true_count = 0
total_sample_count = num_iter * batch_size
step = 0
while step < 10:
    image_batch, label_batch = sess.run([images_test, labels_test])
    predictions = sess.run([top_k_op], feed_dict={image_holder:image_batch,
                                                  label_holder:label_batch})
    true_count += np.sum(predictions)
    step += 1
    print("step: %d" % step)

# 计算最后的计算结果
precision = true_count / total_sample_count
print("precision @ 1 = %.3f" % precision)